# Stock Market API Project

In [1]:
import streamlit as st

In [2]:
api_key = st.secrets["API_KEY"]

# Symbol Search

In [6]:
import requests
import pandas as pd

url = "https://alpha-vantage.p.rapidapi.com/query"

querystring = {
    "datatype": "json",
    "keywords": "State Bank of India",
    "function": "SYMBOL_SEARCH",
}

headers = {"x-rapidapi-key": api_key, "x-rapidapi-host": "alpha-vantage.p.rapidapi.com"}

response = requests.get(url, headers=headers, params=querystring)

data = response.json()["bestMatches"]

res = pd.DataFrame(data)

res

,1. symbol,2. name,3. type,4. region,5. marketOpen,6. marketClose,7. timezone,8. currency,9. matchScore
0,SBIN.BSE,State Bank of India,Equity,India/Bombay,09:15,15:30,UTC+5.5,INR,1.0000
1,SBKFF,State Bank of India,Equity,United States,09:30,16:00,UTC-04,USD,1.0000
2,SID.FRK,State Bank of India,Equity,Frankfurt,08:00,20:00,UTC+02,EUR,1.0000
3,SBID.LON,State Bank of India GDR,Equity,United Kingdom,08:00,16:30,UTC+01,USD,0.9048


### Getting acutal data for SBIN.BSE

In [15]:
url = "https://alpha-vantage.p.rapidapi.com/query"

querystring = {
    "function": "TIME_SERIES_DAILY",
    "symbol": "SBIN.BSE",
    "outputsize": "compact",
    "datatype": "json",
}

headers = {
    "x-rapidapi-key": "c86d19dba2msh0b9382a7f87c6a3p1f2116jsn57da44b7b1b0",
    "x-rapidapi-host": "alpha-vantage.p.rapidapi.com",
}

response = requests.get(url, headers=headers, params=querystring)

data2 = response.json()["Time Series (Daily)"]
res2 = pd.DataFrame(data2).T

# Convert output to float
res2 = res2.astype(float)

# Conert to index to datetime
res2.index = pd.to_datetime(res2.index)

res2

,1. open,2. high,3. low,4. close,5. volume
2025-05-02,792.05,802.95,789.25,800.05,335006.0
2025-04-30,812.85,812.85,784.45,788.15,412886.0
2025-04-29,820.00,829.00,808.75,811.75,699284.0
2025-04-28,796.30,822.50,796.30,817.60,1068737.0
2025-04-25,813.60,819.10,787.10,798.75,383883.0
...,...,...,...,...,...
2024-12-10,862.00,868.85,858.50,867.60,197287.0
2024-12-09,862.95,866.80,856.75,857.85,646144.0
2024-12-06,866.95,875.50,856.85,864.00,431320.0
2024-12-05,861.65,871.00,855.65,864.85,379508.0


In [16]:
res2.dtypes

1. open      float64
2. high      float64
3. low       float64
4. close     float64
5. volume    float64
dtype: object

In [17]:
res2.index

DatetimeIndex(['2025-05-02', '2025-04-30', '2025-04-29', '2025-04-28',
               '2025-04-25', '2025-04-24', '2025-04-23', '2025-04-22',
               '2025-04-21', '2025-04-17', '2025-04-16', '2025-04-15',
               '2025-04-11', '2025-04-09', '2025-04-08', '2025-04-07',
               '2025-04-04', '2025-04-03', '2025-04-02', '2025-04-01',
               '2025-03-28', '2025-03-27', '2025-03-26', '2025-03-25',
               '2025-03-24', '2025-03-21', '2025-03-20', '2025-03-19',
               '2025-03-18', '2025-03-17', '2025-03-13', '2025-03-12',
               '2025-03-11', '2025-03-10', '2025-03-07', '2025-03-06',
               '2025-03-05', '2025-03-04', '2025-03-03', '2025-02-28',
               '2025-02-27', '2025-02-25', '2025-02-24', '2025-02-21',
               '2025-02-20', '2025-02-19', '2025-02-18', '2025-02-17',
               '2025-02-14', '2025-02-13', '2025-02-12', '2025-02-11',
               '2025-02-10', '2025-02-07', '2025-02-06', '2025-02-05',
      

In [18]:
res2.to_csv("SBIN.csv")

### Plotting the candlestick chart